In [ ]:
from pyobsplot import Obsplot, Plot, d3, Math, js
op = Obsplot()

import polars as pl

simpsons = pl.read_csv("data/simpsons.csv")
penguins = pl.read_csv("data/penguins.csv")
metros = pl.read_csv("data/metros.csv")

# simpsons cell plot
op({
  "height": 640,
  "padding": 0.05,
  "grid": True,
  "x": {"axis": "top", "label": "Season"},
  "y": {"label": "Episode"},
  "color": {"type": "linear", "scheme": "PiYG"},
  "marks": [
    Plot.cell(simpsons, {"x": "season", "y": "number_in_season", "fill": "imdb_rating"}),
    Plot.text(simpsons, {"x": "season", "y": "number_in_season", "text": "imdb_rating", "title": "title"})
  ]    
})


In [ ]:
# penguins faceted plot
op({
  "height": 600,
  "grid": True,
  "facet": {
    "data": penguins,
    "x": "sex",
    "y": "species",
    "marginRight": 80
  },
  "marks": [
    Plot.frame({"facet": False}),
    Plot.dot(penguins, {
      "x": "culmen_depth_mm",
      "y": "culmen_length_mm",
      "r": 1.5,
      "fill": "#ccc",
      "facet": "exclude"
    }),
    Plot.dot(penguins, {
       "x": "culmen_depth_mm",
       "y": "culmen_length_mm",
       "facet": True
    })
  ]
})

In [ ]:
# metros arrow plot
op({
  "height": 600,
  "grid": True,
  "inset": 10,
  "x": {
    "type": "log",
    "label": "Population →"
  },
  "y": {
    "label": "↑ Inequality",
    "ticks": 4
  },
  "color": {
    "type": "diverging",
    "scheme": "burd",
    "label": "Change in inequality from 1980 to 2015",
    "legend": True,
    "ticks": 6,
    "tickFormat": "+f"
  },
  "marks": [
    Plot.arrow(metros, {
      "x1": "POP_1980",
      "y1": "R90_10_1980",
      "x2": "POP_2015",
      "y2": "R90_10_2015",
      "bend": True,
      "stroke": js("d => d.R90_10_2015 - d.R90_10_1980")
    }),
    Plot.text(metros, {
      "x": "POP_2015",
      "y": "R90_10_2015",
      "filter": "highlight",
      "text": "nyt_display",
      "fill": "currentColor",
      "stroke": "white",
      "dy": -6
    })
  ]
})

In [ ]:
# stocks map transform plot with d3 call in js()
stocks = pl.read_csv("data/stocks.csv", try_parse_dates=True)

op(
    {
        "marginRight": 40,
        "y": {
            "type": "log",
            "grid": True,
            "label": "↑ Change in price (%)",
            "tickFormat": js('(f => x => f((x - 1) * 100))(d3.format("+d"))'),
        },
        "marks": [
            Plot.ruleY([1]),
            Plot.line(
                stocks, Plot.normalizeY({"x": "Date", "y": "Close", "stroke": "Symbol"})
            ),
            Plot.text(
                stocks,
                Plot.selectLast(
                    Plot.normalizeY(
                        {
                            "x": "Date",
                            "y": "Close",
                            "z": "Symbol",
                            "text": "Symbol",
                            "textAnchor": "start",
                            "dx": 3,
                        }
                    )
                ),
            ),
        ],
    }
)

In [ ]:
# age by state plot
stateage = (
    pl.read_csv("data/us-population-state-age.csv")
    .melt(id_vars="name", variable_name="age", value_name="population")
    .rename({"name": "state"})
)
ages = stateage.get_column("age").unique(maintain_order=True).to_list()
states = (
    stateage.with_columns(
        (pl.col("population") / pl.col("population").sum().over("state")).alias(
            "percent"
        )
    )
    .filter(pl.col("age") == "≥80")
    .sort(pl.col("percent"), descending=True)
    .get_column("state")
    .to_list()
)

xy = {"basis": "sum", "z": "state", "x": "population", "y": "state"}

op(
    {
        "height": 660,
        "grid": True,
        "x": {"axis": "top", "label": "Percent (%) →", "transform": js("d => d * 100")},
        "y": {
            "domain": states,
            "axis": None,
        },
        "color": {"scheme": "spectral", "domain": ages, "legend": True},
        "marks": [
            Plot.ruleX([0]),
            Plot.ruleY(
                stateage,
                Plot.groupY({"x1": "min", "x2": "max"}, Plot.normalizeX(xy)),
            ),
            Plot.dot(
                stateage,
                Plot.normalizeX({**xy, "fill": "age"}),
            ),
            Plot.text(
                stateage,
                Plot.selectMinX(
                    Plot.normalizeX(
                        {
                            **xy,
                            "textAnchor": "end",
                            "dx": -6,
                            "text": "state",
                        }
                    )
                ),
            ),
        ],
    }
)